In [1]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download ocular disease recognition dataset
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

100% 1.62G/1.62G [00:09<00:00, 203MB/s]
100% 1.62G/1.62G [00:09<00:00, 187MB/s]


In [5]:
# Extract the data
import zipfile
zip_ref = zipfile.ZipFile('ocular-disease-recognition-odir5k.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [6]:
# !pip install tensorflow-gpu==2.8.3
# !pip install tensorflow-addons==0.17.1

In [7]:
import os
import numpy as np
import pandas as pd

import random
from time import time
from datetime import datetime

In [8]:
df = pd.read_excel('/content/files/ODIR-5K/ODIR-5K/data.xlsx')
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [9]:
leftEyeKeywords = df['Left-Diagnostic Keywords'].copy()
rightEyeKeywords = df['Right-Diagnostic Keywords'].copy()


leftEyeKeywords = leftEyeKeywords.str.split("，")
rightEyeKeywords = rightEyeKeywords.str.split("，")

In [10]:
# Retrieve unique keywords from left and right diagnostic keywords

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(rightEyeKeywords),
                   columns=mlb.classes_,
                   index=rightEyeKeywords.index)

allDiagnosisLeft = res.columns.to_list()
len(allDiagnosisLeft)

res = pd.DataFrame(mlb.fit_transform(leftEyeKeywords),
                   columns=mlb.classes_,
                   index=leftEyeKeywords.index)

allDiagnosisRight = res.columns.to_list()
len(allDiagnosisRight)

allDiagnosis=list(set(allDiagnosisLeft+allDiagnosisRight))
print("total different keys diagnosis :", len(allDiagnosis))

total different keys diagnosis : 105


In [11]:
# Create a function to sort all keywords into label keywords and list row that contains multiple label

test_df = df.copy()
doubleDiagnosisRow = []

def getKeyDiagnosisSingle(colName):
  keyDiagnosis = []
  global doubleDiagnosisRow
  store = True
  for row in range(len(test_df[colName])):
    store = True
    if test_df[colName][row] == 1:
      for lable in test_df.columns[7:]:
        if lable == colName:
          continue
        if test_df[lable][row] == 1:
          doubleDiagnosisRow.append(row)
          store = False
          break

      if store == True:
        for i in rightEyeKeywords[row]:
          keyDiagnosis.append(i)
        for i in leftEyeKeywords[row]:
          keyDiagnosis.append(i)


  keyDiagnosis = list(set(keyDiagnosis))
  return keyDiagnosis

In [12]:
keyNormal = getKeyDiagnosisSingle(test_df.columns[7])
keyDiabetes = getKeyDiagnosisSingle(test_df.columns[8])
keyGlaucoma = getKeyDiagnosisSingle(test_df.columns[9])
keyCataract = getKeyDiagnosisSingle(test_df.columns[10])
keyAMD = getKeyDiagnosisSingle(test_df.columns[11])
keyHypertension = getKeyDiagnosisSingle(test_df.columns[12])
keyMyopia = getKeyDiagnosisSingle(test_df.columns[13])
keyOtherDisease = getKeyDiagnosisSingle(test_df.columns[14])

labelString = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension', 'Myopia', 'Abnormalities']
keyAll = [keyNormal, keyDiabetes, keyGlaucoma, keyCataract, keyAMD, keyHypertension, keyMyopia, keyOtherDisease]

for i in range(8):
  print(labelString[i], len(keyAll[i]))

Normal 3
Diabetes 10
Glaucoma 5
Cataract 3
AMD 3
Hypertension 1
Myopia 6
Abnormalities 57


In [13]:
# Create permutations of 2 samples from 7 labels
from itertools import permutations

comb = list(permutations(range(1, 8), 2))

In [14]:
# Intersect each label keyword with normal keyword
print("intersect by normal :\n")
for i in range(1,len(keyAll)):
  keyAll[i] = list(set(keyAll[i])-set(keyAll[0]))

for i in range(8):
  print(labelString[i], len(keyAll[i]))

intersect by normal :

Normal 3
Diabetes 8
Glaucoma 3
Cataract 1
AMD 2
Hypertension 1
Myopia 4
Abnormalities 54


In [15]:
# Intersect each label with each other except normal keywords
print("\nintersect by others :\n")
for i in range(len(keyAll)-1):
  for j in range(6):
    k = j+6*i
    keyAll[i+1] = list(set(keyAll[i+1])-set(keyAll[comb[k][1]]))

for i in range(8):
  print(labelString[i], len(keyAll[i]))


intersect by others :

Normal 3
Diabetes 7
Glaucoma 3
Cataract 1
AMD 2
Hypertension 1
Myopia 4
Abnormalities 54


In [16]:
# Create a function to join unique keywords from all label

def getAllRecognizedKey(mkeyAll):
  mallkeyDiagnosis = []
  for i in range(len(mkeyAll)):
    mkeyAll[i] = list(set(mkeyAll[i]))
    mallkeyDiagnosis = mallkeyDiagnosis+list(set(mkeyAll[i]))
  return mallkeyDiagnosis

In [17]:
keyNormal, keyDiabetes, keyGlaucoma, keyCataract, keyAMD, keyHypertension, keyMyopia, keyOtherDisease = keyAll[0], keyAll[1], keyAll[2], keyAll[3], keyAll[4], keyAll[5], keyAll[6], keyAll[7]

allkeyDiagnosis = getAllRecognizedKey(keyAll)
len(allkeyDiagnosis)

75

In [18]:
# Check the number of row that contains multiple label then sort it

doubleDiagnosisRow = list(set(doubleDiagnosisRow))
print("double label row ",len(doubleDiagnosisRow))
doubleDiagnosisRow.sort()

double label row  586


In [19]:
# Find keyword that has not been included in any of label keywords

notlisted = []
listed = False
for row in doubleDiagnosisRow:
  # print(row)
  for ilist in leftEyeKeywords[row]:
    # print(ilist)
    listed = False
    for j in keyAll:
      if ilist in j:
        listed = True
        break
    if listed == False:
      notlisted.append(ilist)

for row in doubleDiagnosisRow:
  for ilist in rightEyeKeywords[row]:
    listed = False
    for j in keyAll:
      if ilist in j:
        listed = True
        break
    if listed == False:
      notlisted.append(ilist)

notlisted = list(set(notlisted))
# notlisted
print("not listed diagnosis key :",len(notlisted))

not listed diagnosis key : 30


In [20]:
# Create a function to separate keyword from multiple label row

def intersectFromMultiLabel(mkeyAll):
  mnotRecognizedList = []
  mallkeyDiagnosis = []
  for i in range(len(mkeyAll)):
    mkeyAll[i] = list(set(mkeyAll[i]))
    mallkeyDiagnosis = mallkeyDiagnosis+list(set(mkeyAll[i]))
  for row in doubleDiagnosisRow:
    notlistedList = []
    listedList = []
    colIndex = []
    ind = []
    tempList = []
    for ilist in leftEyeKeywords[row]:
      if ilist not in mallkeyDiagnosis:
        tempList.append(ilist)
    for ilist in rightEyeKeywords[row]:
      if ilist not in mallkeyDiagnosis:
        tempList.append(ilist)

    for i in range(7, len(test_df.columns)):
      if test_df[test_df.columns[i]][row] == 1:
        colIndex.append(i-7)
    tempList = list(set(tempList))
    isContainAbnormal = 7 in colIndex
    if len(tempList) > 0:
      ind = colIndex
      for ilist in leftEyeKeywords[row]:
        if ilist not in tempList:
          listedList.append(ilist)
      for ilist in rightEyeKeywords[row]:
        if ilist not in tempList:
          listedList.append(ilist)

      for ilist in listedList:
        for i in colIndex:
          if ilist in keyNormal:
            continue
          if ilist in mkeyAll[i]:
            ind.remove(i)

      if len(ind) == 0 and isContainAbnormal:
        ind.append(7)
      if len(ind) == 1 and len(tempList) == 1:
        mkeyAll[ind[0]] = mkeyAll[ind[0]] + tempList
        mkeyAll[ind[0]] = list(set(mkeyAll[ind[0]]))
      else:
        print("not recognize")
        mnotRecognizedList.append(tempList[0])
        mnotRecognizedList = list(set(mnotRecognizedList))

    mallkeyDiagnosis = []
    for i in mkeyAll:
      mallkeyDiagnosis = mallkeyDiagnosis+list(set(i))
  return mkeyAll, mnotRecognizedList

In [21]:
itterate = True
notRecognizedList = []
while itterate :
  temp_allkeyDiagnosis = allkeyDiagnosis.copy()
  keyAll, notRecognizedList = intersectFromMultiLabel(keyAll)
  allkeyDiagnosis = getAllRecognizedKey(keyAll)
  # print(len(temp_allkeyDiagnosis), len(allkeyDiagnosis))
  print(notRecognizedList)
  if len(temp_allkeyDiagnosis) == len(allkeyDiagnosis):
    print(True)
    itterate = False


not recognize
not recognize
not recognize
['suspected cataract', 'myopia retinopathy', 'image offset']
not recognize
not recognize
['suspected cataract', 'image offset']
not recognize
not recognize
['suspected cataract', 'image offset']
True


In [22]:
allkeyDiagnosis = getAllRecognizedKey(keyAll)

keyNormal, keyDiabetes, keyGlaucoma, keyCataract, keyAMD, keyHypertension, keyMyopia, keyOtherDisease = keyAll[0], keyAll[1], keyAll[2], keyAll[3], keyAll[4], keyAll[5], keyAll[6], keyAll[7]

for i in range(8):
  print(labelString[i], len(keyAll[i]))

print("\nall recognized key :",len(allkeyDiagnosis))
print("\nnot recognized key : ", list(set(allDiagnosis)-set(allkeyDiagnosis)))

Normal 3
Diabetes 11
Glaucoma 3
Cataract 1
AMD 3
Hypertension 1
Myopia 5
Abnormalities 76

all recognized key : 103

not recognized key :  ['suspected cataract', 'image offset']


In [23]:
# Add keyword manually into one label keywords

string = 'suspected cataract'
if string in notRecognizedList and string not in allkeyDiagnosis:
  keyAll[3].append(string)
  notRecognizedList.remove(string)


In [24]:
# Remove keyword that contains "lens dust", "optic disk photographically invisible"
# "low image quality" , "no fundus image" or "image offset"

for label in keyAll[:]:
  for key in label[:]:
    if 'lens dust' in key or 'optic disk photographically invisible' in key or \
    'low image quality' in key or 'image offset' in key or 'no fundus image' in key:
      label.remove(key)

In [25]:
# Create a dataframe containing image filename, keywords and its label

filename = df['Left-Fundus'].values.tolist() + df['Right-Fundus'].values.tolist()
keywords = leftEyeKeywords.values.tolist() + rightEyeKeywords.values.tolist()

# Assign label for each image filename
keywords = leftEyeKeywords.values.tolist() + rightEyeKeywords.values.tolist()

labels = []
for keyword in keywords:
  temp = []
  for key in keyword:
    if key in keyNormal:
      temp.append('N')
      continue
    elif key in keyDiabetes:
      temp.append('D')
      continue
    elif key in keyGlaucoma:
      temp.append('G')
      continue
    elif key in keyCataract:
      temp.append('C')
      continue
    elif key in keyAMD:
      temp.append('A')
      continue
    elif key in keyHypertension:
      temp.append('H')
      continue
    elif key in keyMyopia:
      temp.append('M')
      continue
    elif key in keyOtherDisease:
      temp.append('O')
      continue
  labels.append(temp)

new_df = pd.DataFrame(list(zip(filename, keywords, labels)), columns =['filename', 'keywords', 'label'])

In [26]:
# Remove row that contains keyword such as "lens dust", "optic disk photographically invisible"
# "low image quality" or "image offset"

index = []
for idx, row in new_df.iterrows():
  for key in row['keywords']:
    if 'lens dust' in key or 'optic disk photographically invisible' in key or \
    'low image quality' in key or 'image offset' in key:
      index.append(idx)

print(len(index))
new_df = new_df.drop(index)
new_df = new_df.reset_index(drop=True)

435


In [27]:
new_df

,filename,keywords,label
0,0_left.jpg,[cataract],[C]
1,1_left.jpg,[normal fundus],[N]
2,2_left.jpg,"[laser spot, moderate non proliferative retino...","[O, D]"
3,3_left.jpg,[normal fundus],[N]
4,4_left.jpg,[macular epiretinal membrane],[O]
...,...,...,...
6563,4686_right.jpg,[proliferative diabetic retinopathy],[D]
6564,4688_right.jpg,[moderate non proliferative retinopathy],[D]
6565,4689_right.jpg,[normal fundus],[N]
6566,4690_right.jpg,[mild nonproliferative retinopathy],[D]


In [28]:
new_df.to_json("output.json", orient="records")